In [322]:
# -*- coding: utf-8 -*-
"""
Created on Mon Aug 15 14:49:58 2022

@author: Sidi Wu and Cédric Beaulac

Functional autoencoder implementation
"""

# Import modules
import sys
import torch
import torch.nn.init as init
import torch.nn.functional as F
import torch.nn as nn
from torch import optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from torchvision.utils import save_image
import pandas as pd
import numpy as np
from numpy import *
import seaborn as sns
import matplotlib
#matplotlib.use('TkAgg')
from matplotlib import pyplot as plt
# import skfda as fda
# from skfda import representation as representation
# from skfda.exploratory.visualization import FPCAPlot
# # from skfda.exploratory.visualization import FPCAPlot
# # from skfda.preprocessing.dim_reduction import FPCA
# # from skfda.representation.basis import BSpline, Fourier, Monomial
import scipy
from scipy.interpolate import BSpline
import os
import ignite
import sklearn
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.cluster import KMeans
import random
from random import seed
from scipy import stats
import statistics
from statistics import stdev
import skfda
from skfda.representation import FDataGrid as fd
from skfda.representation.basis import BSpline as bsp
from skfda.representation.basis import FDataBasis as bsp_reconstruction




In [323]:
##Importation des données sous forme FDA

ElNino_OISST_region_1and2=skfda.datasets.fetch_cran("ElNino_OISST_region_1and2","rainbow")
ElNino_OISST_region_3=skfda.datasets.fetch_cran("ElNino_OISST_region_3","rainbow")
ElNino_OISST_region_4=skfda.datasets.fetch_cran("ElNino_OISST_region_4","rainbow")
ElNino_OISST_region_3and4=skfda.datasets.fetch_cran("ElNino_OISST_region_3and4","rainbow")


c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "sfts". The constructor for class "fts" will be used instead.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "fts". The constructor for class "fds" will be used instead.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "fds". The underlying R object is returned instead.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "sfts". The constructor for class "fts" will be used instead.
  warnings.warn(
c:\Users\Utilisateur\anaconda3\lib\site-packages\rdata\conversion\_conversion.py:843: UserWarning: Missing constructor for R class "fts". The constructor for class "fds" w

In [60]:
ElNino_OISST_region_1and2_y=ElNino_OISST_region_1and2['ElNino_OISST_region_1and2']['y'].to_numpy()
ElNino_OISST_region_1and2_x=ElNino_OISST_region_1and2['ElNino_OISST_region_1and2']['x']
# plt.plot(ElNino_OISST_region_1and2_x,ElNino_OISST_region_1and2_y)
# plt.show(
# )
ElNino_OISST_region_3_y=ElNino_OISST_region_3['ElNino_OISST_region_3']['y'].to_numpy()
ElNino_OISST_region_3_x=ElNino_OISST_region_3['ElNino_OISST_region_3']['x']
# plt.plot(ElNino_OISST_region_3_x,ElNino_OISST_region_3_y)
# plt.show(

# )
ElNino_OISST_region_3and4_y=ElNino_OISST_region_3and4['ElNino_OISST_region_3and4']['y'].to_numpy()
ElNino_OISST_region_3and4_x=ElNino_OISST_region_3and4['ElNino_OISST_region_3and4']['x']
# plt.plot(ElNino_OISST_region_3and4_x,ElNino_OISST_region_3and4_y)
# plt.show(
# )


ElNino_OISST_region_4_y=ElNino_OISST_region_4['ElNino_OISST_region_4']['y'].to_numpy()
ElNino_OISST_region_4_x=ElNino_OISST_region_4['ElNino_OISST_region_4']['x']
# plt.plot(ElNino_OISST_region_4_x,ElNino_OISST_region_4_y)
# plt.show(
# )
grid_points=ElNino_OISST_region_1and2_x


In [61]:
x0=torch.tensor(ElNino_OISST_region_1and2_y).float().cuda()
x1=torch.tensor(ElNino_OISST_region_3_y).float().cuda()
x2=torch.tensor(ElNino_OISST_region_4_y).float().cuda()
x3=torch.tensor(ElNino_OISST_region_3and4_y).float().cuda()
x0=torch.t(x0)
x1=torch.t(x1)
x2=torch.t(x2)
x3=torch.t(x3)

# train_index=np.random.permutation(torch.tensor(np.arange(30)))
# labels = torch.tensor([0] * x0.shape[0] + [1] * x1.shape[0] + [2] * x2.shape[0] + [3] * x3.shape[0])
# x0_train,y0_train=x0[train_index,:],labels[train_index]
# x1_train,y1_train=x1[train_index,:],labels[train_index]
# x2_train,y2_train=x2[train_index,:],labels[train_index]
# x3_train,y3_train=x3[train_index,:],labels[train_index]
# X=torch.concatenate([x0_train,x1_train,x2_train,x3_train],axis=0)
# X_test=torch.concatenate([x0[~train_index],x1[~train_index],x2[~train_index],x3[~train_index]],axis=0)
# Y_test=torch.tensor(labels[~train_index_long])
# Y_test

In [62]:

labels = torch.tensor([0] * x0.shape[0] + [1] * x1.shape[0] + [2] * x2.shape[0] + [3] * x3.shape[0])
X=torch.concatenate([x0,x1,x2,x3])
x_train,x_test,y_train,y_test=sklearn.model_selection.train_test_split(X,labels,shuffle=True)
x_train=x_train.reshape(111,1,12).cuda()
y_train=y_train.unsqueeze(1).cuda()
x_test=x_test.reshape(37,1,12).cuda()
y_test=y_test.unsqueeze(1).cuda()




In [196]:

def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 3)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 3)
        torch.nn.init.constant_(m.bias.data, 0.0)


In [329]:
class FCNN(nn.Module):
    def __init__(self):
        super(FCNN,self).__init__()
        self.convlayer1=nn.Sequential(

            nn.Conv1d(1,64,kernel_size=5,stride=1,padding=1,dilation=1),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(0.18),
        )
        
        self.convlayer2=nn.Sequential(

            nn.Conv1d(64,128,kernel_size=5,stride=1,padding=1,dilation=1),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.18),
        )

        self.fc_block=nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*8,128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.18),
        
            nn.Linear(128,4),
        )

    def forward(self,x):
        Conv_out=self.convlayer1(x)
        Conv_out2=self.convlayer2(Conv_out)
        n_conv_out=Conv_out2.shape[1]
        Lin_out=self.fc_block(Conv_out2)
        return Lin_out.float().unsqueeze_(2).unsqueeze_(3)


In [393]:
n_conv_out=10
CNN=FCNN().cuda().apply(weights_init_normal)
Conv_out=CNN.convlayer1(x_train)
[x_train.shape,Conv_out.shape]


[torch.Size([111, 1, 12]), torch.Size([111, 64, 10])]

In [394]:
lr=0.00001
batch_size=32
betas=[0.45,0.98]
optimizer=optim.Adam(params=CNN.parameters(),lr=lr,betas=betas)

loss_function=nn.CrossEntropyLoss()
train_loss=torch.tensor(0).cuda().long()


In [395]:

from tqdm import tqdm
def train(n_epochs):
    train_loss=torch.tensor(0).cuda().long()
    
    for i in range(int(len(x_train)/batch_size)):
        functions_train=x_train[batch_size*i:batch_size*(1+i),:,:]
        labels_train=y_train[batch_size*i:batch_size*(1+i)]
        optimizer.zero_grad()
        output=CNN(functions_train)
        loss=loss_function(input=output,target=labels_train.unsqueeze(1).long())
        
        loss.backward()
        optimizer.step()
        train_loss+=loss.long()
    return train_loss,loss 

        


    

In [453]:
train(100)
print("Précision moyenne =",((torch.sum(torch.argmax(CNN(x_test),dim=1).squeeze(1)==y_test)/37)*100).detach().cpu().numpy(),"%")

Précision moyenne = 89.189186 %


In [307]:
import gc
import torch
torch.cuda.empty_cache()
gc.collect()

715